In [1]:
# Import libraries
import pandas as pd
import numpy as np

# Plotting libraries
import cufflinks as cf
cf.set_config_file(offline=True)

In [2]:
# specify lanbda
lambda_ = 0.05

# specify tenors
tenor = np.arange(15)

# calculate survival probability
data = pd.DataFrame({'Survival': np.exp(-lambda_*tenor)})
data['Default'] = 1. - data['Survival']

In [3]:
# Visualize the plot
data.iplot(title='Survival vs Default Probability with Constante Lambda',
           xTitle='CDS Maturity',
           yTitle='Survival Probability',
           secondary_y='Default',
           secondary_y_title='Default Probability')

In [4]:
# specify lambdas
lambda1 = 0.01
lambda2 = 0.05
lambda3 = 0.10

# Subsume into dataframes
data_ = pd.DataFrame({'$\lambda = 1\%$': np.exp(-lambda1*tenor)})
data_['$\lambda = 5\%$'] = np.exp(-lambda2*tenor)
data_['$\lambda = 10\%$'] = np.exp(-lambda3*tenor)
data_.iplot(title='Survival Probabilities',
            xTitle='CDS Maturity',
            yTitle='Survival Probability')

In [5]:
# Read the cds spreads data
df = pd.read_csv('pythonlab21-data.txt', sep='\t')
df

,Maturity,Df,Spread
0,0,0.00,0.0
1,1,0.97,50.0
2,2,0.94,77.0
3,3,0.92,94.0
4,4,0.89,109.5
5,5,0.86,125.0


In [6]:
# Convert bps to decimal
df['Spread'] = df['Spread']/1000

# Specify delta t
df['Dt'] = df['Maturity'].diff().fillna(0)

# Specify recovery rate
RR = 0.40

# Loss rate
L = 1.0 - RR

df

,Maturity,Df,Spread,Dt
0,0,0.00,0.0000,0.0
1,1,0.97,0.0500,1.0
2,2,0.94,0.0770,1.0
3,3,0.92,0.0940,1.0
4,4,0.89,0.1095,1.0
5,5,0.86,0.1250,1.0


In [7]:
# initialize the variables
term = term1 = term2 = divider = 0

for i in range(0, len(df.index)):
    if i == 0: df.loc[i,'Survival'] = 1
    if i == 1: df.loc[i,'Survival'] = L / (L+df.loc[i,'Dt']*df.loc[i,'Spread'])
    if i > 1:
        terms = 0
        for j in range(1, i):
            term = df.loc[j,'Df']*(L*df.loc[j-1,'Survival'] - \
                                          (L + df.loc[j,'Dt']*df.loc[i,'Spread'])* \
                                          df.loc[j,'Survival'])
            terms = terms + term
        
        divider = df.loc[i,'Df']*(L+df.loc[i,'Dt']*df.loc[i,'Spread'])
        term1 = terms/divider

        term2 = (L*df.loc[i-1,'Survival']) / (L + (df.loc[i,'Dt'] * df.loc[i,'Spread']))
        
        df.loc[i,'Survival'] = term1 + term2

# derive probability of default
df['Default'] = 1 - df['Survival']

# output the results
df

,Maturity,Df,Spread,Dt,Survival,Default
0,0,0.00,0.0000,0.0,1.000000,0.000000
1,1,0.97,0.0500,1.0,0.923077,0.076923
2,2,0.94,0.0770,1.0,0.780100,0.219900
3,3,0.92,0.0940,1.0,0.631073,0.368927
4,4,0.89,0.1095,1.0,0.479447,0.520553
5,5,0.86,0.1250,1.0,0.331255,0.668745


In [8]:
# plot survival probability
df[['Survival', 'Default']].iplot(title='CDS Bootsrapping',
                                  xTitle='CDS Maturity', yTitle='Survival Probability',
                                  secondary_y = 'Default', secondary_y_title='Default Probability')

In [9]:
def survival_probability(maturity, discountfactor, spread, recovery, is_plot=False):
    
    # subsume list of inputs into a dataframe
    df = pd.DataFrame({'Maturity': maturity, 'Df': discountfactor, 'Spread': spread})
    
    # convert bps to decimal
    df['Spread'] = df['Spread']/10000

    # specify delta_t
    df['Dt'] = df['Maturity'].diff().fillna(0)

    # loss rate
    L = 1.0 - recovery
    
    # initialize the variables
    term = term1 = term2 = divider = 0
    
    for i in range(0, len(df.index)):
        if i == 0: df.loc[i,'Survival'] = 1
        if i == 1: df.loc[i,'Survival'] = L / (L+df.loc[i,'Dt']*df.loc[i,'Spread'])
        if i > 1:
            terms = 0
            for j in range(1, i):
                term = df.loc[j,'Df']*(L*df.loc[j-1,'Survival'] - \
                                              (L + df.loc[j,'Dt']*df.loc[i,'Spread'])* \
                                              df.loc[j,'Survival'])
                terms = terms + term  
           
            divider = df.loc[i,'Df']*(L+df.loc[i,'Dt']*df.loc[i,'Spread'])
            term1 = terms/divider

            term2 = (L*df.loc[i-1,'Survival']) / (L + (df.loc[i,'Dt'] * df.loc[i,'Spread']))

            df.loc[i,'Survival'] = term1 + term2

    # derive probability of default
    df['Default'] = 1. - df['Survival']
    
    if is_plot:
        # plot survival probability
        df[['Survival', 'Default']].iplot(title='Survival vs Default Probability', 
                                          xTitle='CDS Maturity', 
                                          yTitle='Survival Probability', 
                                          secondary_y = 'Default', 
                                          secondary_y_title='Default Probability')

    return df

In [10]:
# inputs
maturity = np.arange(6)
discountfactor = [0, 0.97, 0.94, 0.92, 0.89, 0.86]
cds_spread = [0, 50, 77, 94, 109.5, 125]
recovery = 0.40

# call the user defined function
sp = survival_probability(maturity,discountfactor,cds_spread,recovery,is_plot=False)
sp

,Maturity,Df,Spread,Dt,Survival,Default
0,0,0.00,0.00000,0.0,1.000000,0.000000
1,1,0.97,0.00500,1.0,0.991736,0.008264
2,2,0.94,0.00770,1.0,0.974623,0.025377
3,3,0.92,0.00940,1.0,0.953894,0.046106
4,4,0.89,0.01095,1.0,0.928942,0.071058
5,5,0.86,0.01250,1.0,0.899443,0.100557


In [11]:
# inputs
maturity = np.arange(6)
discountfactor = [0, 0.9803, 0.9514, 0.9159, 0.8756, 0.8328]
cds_spread = [0, 29, 39, 46, 52, 57]
recovery = 0.50

# call the user defined function
survival_probability(maturity,discountfactor,cds_spread,recovery,is_plot=False)

,Maturity,Df,Spread,Dt,Survival,Default
0,0,0.0000,0.0000,0.0,1.000000,0.000000
1,1,0.9803,0.0029,1.0,0.994233,0.005767
2,2,0.9514,0.0039,1.0,0.984505,0.015495
3,3,0.9159,0.0046,1.0,0.972636,0.027364
4,4,0.8756,0.0052,1.0,0.958824,0.041176
5,5,0.8328,0.0057,1.0,0.943693,0.056307


In [12]:
# HSBC 
maturity = np.arange(6)
discountfactor = [0, 0.9972, 0.9916, 0.9775, 0.9619, 0.9426]
cds_spread = [0, 11.2, 27.7, 36.9, 57.1, 67.8]
recovery = 0.40

# call function
hsbc = survival_probability(maturity,discountfactor,cds_spread,recovery,is_plot=False)
hsbc

,Maturity,Df,Spread,Dt,Survival,Default
0,0,0.0000,0.00000,0.0,1.000000,0.000000
1,1,0.9972,0.00112,1.0,0.998137,0.001863
2,2,0.9916,0.00277,1.0,0.990802,0.009198
3,3,0.9775,0.00369,1.0,0.981663,0.018337
4,4,0.9619,0.00571,1.0,0.962224,0.037776
5,5,0.9426,0.00678,1.0,0.944246,0.055754


In [13]:
# Barclays 
maturity = np.arange(6)
discountfactor = [0, 0.9972, 0.9916, 0.9775, 0.9619, 0.9426]
cds_spread = [0, 17.7, 44.6, 54.8, 83.5, 96.2]
recovery = 0.40

# call function
barclays = survival_probability(maturity,discountfactor,cds_spread,recovery,is_plot=False)
barclays

,Maturity,Df,Spread,Dt,Survival,Default
0,0,0.0000,0.00000,0.0,1.000000,0.000000
1,1,0.9972,0.00177,1.0,0.997059,0.002941
2,2,0.9916,0.00446,1.0,0.985240,0.014760
3,3,0.9775,0.00548,1.0,0.972925,0.027075
4,4,0.9619,0.00835,1.0,0.945239,0.054761
5,5,0.9426,0.00962,1.0,0.921855,0.078145


In [14]:
# create dataframe of banks cds
bank = pd.DataFrame({'Barclays CDS': barclays['Spread']*10000,
                     'HSBC CDS': hsbc['Spread']*10000,
                     'Barclays PD': barclays['Default']*100,
                     'HSBC PD': hsbc['Default']*100})

# plot bank cds & probability of default
bank.iplot(title='Spread vs Default Probability',
           xTitle='CDS Maturity', 
           yTitle='CDS Spread (bps)', 
           secondary_y = ['Barclays PD', 'HSBC PD'],
           secondary_y_title='Default Probability (%)')

In [15]:
def get_cds_spread(maturity, discountfactor, probability, recovery, is_plot=False):
    
    # subsume list of inputs into a dataframe
    df = pd.DataFrame({'Maturity': maturity, 'Df': discountfactor, 'Survival': probability})
    
    # specify delta_t
    df['Dt'] = df['Maturity'].diff().fillna(0)

    # loss rate
    L = 1.0 - recovery
    
    # initialize the variables
    nterm = dterm = 0
    
    for i in range(0, len(df.index)):
        if i == 0: df.loc[i,'Spread'] = 0
        else:
            nterms = 0; dterms = 0
            for j in range(1, i+1):
                nterm = L * df.loc[j,'Df']*(df.loc[j-1,'Survival'] - df.loc[j,'Survival'])
                nterms = nterms + nterm
                
                dterm = df.loc[j,'Df'] * df.loc[j,'Survival'] * df.loc[j,'Dt']
                dterms = dterms + dterm
                
            df.loc[i,'Spread'] = nterms / dterms * 10000
            
    if is_plot:
        # plot spreads
        df['Spread'][1:].iplot(kind='bar', 
                           title='CDS Spreads',
                           xTitle='CDS Maturity', 
                           yTitle='CDS Spreads (bps)')
        
    return df

In [16]:
# inputs
maturity = np.arange(6)
discountfactor = [0, 0.97, 0.94, 0.92, 0.89, 0.86]
cds_spread = [0, 50, 77, 94, 109.5, 125]
recovery = 0.40

get_cds_spread(maturity, discountfactor, sp.Survival, recovery, is_plot=True)

,Maturity,Df,Survival,Dt,Spread
0,0,0.00,1.000000,0.0,0.0
1,1,0.97,0.991736,1.0,50.0
2,2,0.94,0.974623,1.0,77.0
3,3,0.92,0.953894,1.0,94.0
4,4,0.89,0.928942,1.0,109.5
5,5,0.86,0.899443,1.0,125.0
